![University of Tehran](./img/UT.png)
#   <font color='red'><center>AI CA 2 - GA<center></font> 
## <center>Dr. Fadaei<center>
### <center>Daniyal Maroufi<center>
### <center>810098039<center>

## Aim

This assignment aims to use the Genetic Algorithm (GA) to find a decoded text's encryption key (14 characters). We prepared a global text to generate the dictionary, and we will use it to decipher the text.

# Problem Description

In this assignment, we are trying to decode an encoded text without knowing the encryption key. We should find the encryption key using the genetic algorithm to decipher the text. 
First, we read a long global text and generate a dictionary of the words, and we are assured that all of the words in the original text(that we want to decode) are in this dictionary. Then, we guess a random encryption key and decrypt the text. If all the words are present in the extracted dictionary, we claim that the supposed key is correct and the target text is decoded successfully.

# Algorithm Description

First, we generate 300 possible answers (chromosomes) as the initial population or the first generation. Then, we use a fitness function to rank the chromosomes. The fitness function presents how good each chromosome (solution) is. By comparing all chromosomes, we can find the best chromosomes of the generations. Then we will create a new generation based on these selected chromosomes using crossover and mutation processes. By repeating this process, we will finally find the solution that satisfies the expectations of the solution.



# Algorithm Implementation

## Part 0 - Generating the Dictionary

In this part, we perform the text data cleaning process, which contains contraction replacement, converting the words to lower case, and removing the stopwords and all punctuation. For eliminating the stopwords, we use the nltk python library.


## Part 1 - Definition of Genes and Chromosome

Each chromosome represents a possible solution to the problem in the Genetic Algorithm. Hence, we define the chromosome as the encryption key consisting of 14 genes (characters). Then we use the chromosomes to evolve through the algorithm process, and finally, we can find the correct key.


## Part 2 - Generating Initial Population

We generate 300 random chromosomes as the initial population.


## Part 3 - Ranking Population

To rank the population, we decode the encoded text with each chromosome and count the number of words of the decoded text that exist in the dictionary. If all the decoded words exist in the dictionary, we can conclude that the encryption key is found.


In [4]:
import nltk
import time
import string
import random
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /home/dani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
class Decoder():

    population=[]

    def __init__(self, globalText, encodedText, keyLength=14, InitialPopulationSize=300):
        self.global_text=globalText
        self.dictionary=self.generateDictionary()
        self.encoded_text=' '.join(self.cleanText(encodedText))
        self.key_length=keyLength
        self.initial_population_size=InitialPopulationSize


    def generateDictionary(self):
        self.dictionary=self.cleanText(self.global_text)

    def cleanText(self,text):
        Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
        "'d":" would","'ve":" have","'re":" are"}

        # replace contractions
        for key,value in Apos_dict.items():
            if key in text:
                text=text.replace(key,value)

        text=text.lower()

        # remove punctuations
        for letter in text:
            if letter not in string.ascii_lowercase:
                text=text.replace(letter,' ')
        for punctuation in string.punctuation:
            text=text.replace(punctuation,' ')

        text=text.split()

        dictionary=[]

        for word in text:
            if word not in stopwords.words('english'):
                dictionary.append(word)
        return dictionary

    def decryptLetter(self,e,k):
        e=list(string.ascii_lowercase).index(e)
        k=list(string.ascii_lowercase).index(k)
        return string.ascii_lowercase[(e-k+26)%26]

    def generateChromosome(self):
        return ''.join(random.choice(string.ascii_lowercase) for _ in range(self.key_length))

    def generateInitialPopulation(self):
        for _ in range(self.initial_population_size):
            self.population.append(self.generateChromosome())

    def repeatKey(self,key,desiredLength):
        return key*int(desiredLength/len(key))+key[:desiredLength%len(key)]

    def decrypt(self,key):
        decoded_text=''
        repeated_key=self.repeatKey(key)
        for idx,letter in enumerate(self.encoded_text):
            decoded_text+=self.decryptLetter(letter,key[idx])
        return decoded_text

    def checkPresence(self,word):
        if word in self.dictionary:
            return True
        return False

    def countPresence(self, decoded_text):
        presence=0
        for word in decoded_text:
            if self.checkPresence(word):
                presence+=1
        return presence

    def calculateFitness(self, chromosome):
        decoded_text=self.decrypt(chromosome)
        return self.countPresence(decoded_text)

    def rankPopulation(self):
        pass

    def crossOver(self, chromosome1, chromosome2):
        pass

    def mutate(self, chromosome):
        pass

    def generateNewPopulation(self):
        pass

    def decode(self):
        pass


In [48]:
encodedText = open('./Resources/encoded_text.txt').read()
globalText = open('./Resources/global_text.txt').read()
d = Decoder(globalText, encodedText)
# decodedText = d.decode()
d.generateDictionary()
len(d.dictionary)
d.generateInitialPopulation()
# print(d.population)
print(d.decryptLetter('n','r'))
print(d.repeatKey('abc',11))
# print(d.decrypt())



w
abcabcabcab
